In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.autograd import Function
from torchsample.transforms import RangeNormalize
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import gradcheck


# Sample Data

In [2]:
def rangeNormalize(data):
    return 0.99*(data-torch.min(data))/(torch.max(data)-torch.min(data))

In [3]:
# data size: batch x num_points x feature
nbins = 4
batch_size = 3
num_points = 5
feature_size = 3
test_data = np.random.random_sample((batch_size, num_points, feature_size))

test_data = rangeNormalize(torch.from_numpy(test_data)).numpy()
print(test_data)

[[[ 0.92577792  0.97461713  0.26061324]
  [ 0.27619581  0.17157076  0.13439798]
  [ 0.          0.46275909  0.59747843]
  [ 0.84886718  0.18526368  0.7580244 ]
  [ 0.69606309  0.99        0.49138517]]

 [[ 0.28175641  0.60825135  0.21435954]
  [ 0.06277362  0.14309147  0.12175607]
  [ 0.5701884   0.1956856   0.25646536]
  [ 0.49377944  0.98901034  0.93569606]
  [ 0.72656384  0.4124462   0.75128254]]

 [[ 0.05954154  0.40998872  0.43326102]
  [ 0.32730437  0.58881543  0.90616193]
  [ 0.88365218  0.39360379  0.37034296]
  [ 0.14999468  0.74478583  0.27319051]
  [ 0.15413811  0.0761583   0.08925235]]]


# Histogram 1d

In [4]:
def histogram(data, bins, range=(0,1)):
    """
    find the histogram of given data
    Current range, (0,1), others haven't been tested
    
    return histogram, TODO: ret_index, ret_value
    
    """
    ret_histogram = torch.zeros(bins)
    ret_indexes = torch.zeros(data.size())
    for idx, d in enumerate(data.view(data.numel())):
        if d == 1.0:
            bin_number = bins - 1
        else:
            bin_number = int(bins*d/(range[1]-range[0]))
        if bin_number == bins:
            bin_number = bin_number - 1
        
        ret_histogram[bin_number] += 1
        if len(data.size()) == 1:
            ret_indexes[idx] = bin_number
        else:
            idx_a = int(idx/data.size()[-1])
            idx_b = idx%data.size()[-1]
            ret_indexes[idx_a, idx_b] = bin_number
    
    return ret_histogram, ret_indexes

# Histogram1d Vectorize

In [5]:
def histogram_vectorize(data, bins, range=(0,1)):
    """
    find the histogram of given data: batch x seq x feature
    Current range, (0,1), others haven't been tested
    
    return histogram, TODO: ret_index, ret_value
    
    """
    ret_histogram = torch.zeros(bins)
    ret_indexes = torch.zeros(data.size())
    
    # calculate the the corresponding bin number 
    temp = (data*bins).int()
    
    # transform into one-hot vector and then sum
    y_tensor = temp
    y_tensor = y_tensor.type(torch.LongTensor).view(data.size()[0],-1, 1)
    # given a list of numbers, we transform each number {i} to one_hot vector(nbins x 1) where index i is 1
    # scatter(dim, index, val)
    y_one_hot = torch.zeros(data.size()[0],y_tensor.size()[1], bins).scatter_(2, y_tensor, 1)
    
    ret_indexes = temp.view(data.size())
    ret_histogram = y_one_hot.sum(1)

    return ret_histogram, ret_indexes
histogram_vectorize(torch.from_numpy(test_data), nbins)

(
  4  4  2  5
  5  4  3  3
  5  6  2  2
 [torch.FloatTensor of size 3x4], 
 (0 ,.,.) = 
   3  3  1
   1  0  0
   0  1  2
   3  0  3
   2  3  1
 
 (1 ,.,.) = 
   1  2  0
   0  0  0
   2  0  1
   1  3  3
   2  1  3
 
 (2 ,.,.) = 
   0  1  1
   1  2  3
   3  1  1
   0  2  1
   0  0  0
 [torch.IntTensor of size 3x5x3])

# Testing output (with numpy)

In [6]:
# mirror output


def np_solution(x,nbins):
    histograms = np.zeros((batch_size, nbins))
    for i in range(batch_size):
        # assert sum is the same as the count
        histograms[i,:] = np.histogram(x[i], bins=nbins, range=(0,1))[0]
    return histograms

def my_histogram(x,nbins):
    histograms = torch.zeros((batch_size, nbins))
    histograms_indexes = torch.zeros(x.shape)
    for i in range(batch_size):
        histograms[i,:], histograms_indexes[i,:] = histogram(torch.from_numpy(x[i]), bins=nbins)
    return histograms, histograms_indexes

def my_histogram_vec(x,nbins):
    my_histogram_vec, my_indexes_vec = histogram_vectorize(torch.from_numpy(x), bins=nbins, range=(0,1))
    return my_histogram_vec, my_indexes_vec

np_hist = np_solution(test_data,nbins)
my_hist, my_idx = my_histogram(test_data,nbins)
my_hist_v, my_idx_vec = my_histogram_vec(test_data,nbins)

assert(np.all(np_hist==my_hist))
assert(np.all(np_hist==my_hist_v))
assert(np.all(my_idx.float() == my_idx_vec.float()))
     


# Testing Time

In [7]:
import time

print("np solution time")
t = time.time()
np_hist = np_solution(test_data, nbins)
print(time.time() - t)

print("my hist time")
t = time.time()
my_hist = my_histogram(test_data, nbins)
print(time.time() - t)

print("my hist vec time")
t = time.time()
my_hist_v = my_histogram_vec(test_data, nbins)
print(time.time() - t)



np solution time
0.00029921531677246094
my hist time
0.0007569789886474609
my hist vec time
0.00021958351135253906


# Histogram 2d Vectorize

In [8]:
def histogram2d_vectorize(data, bins, range=(0,1)):
    """
    find the histogram of given data: batch x seq x feature
    Current range, (0,1), others haven't been tested
    
    return histogram along the second column. eg : batch x bins x feature
    
    """
    ret_histogram = torch.zeros(bins)
    ret_indexes = torch.zeros(data.size())
    
    # calculate the the corresponding bin number 
    temp = (data*bins).int()
    temp = temp.transpose(2,1)
    # transform into one-hot vector and then sum
    y_tensor = temp
    y_tensor = y_tensor.type(torch.LongTensor).view(data.size()[0], data.size()[-1],-1, 1)

    # given a list of numbers, we transform each number {i} to one_hot vector(nbins x 1) where index i is 1
    # scatter(dim, index, val)
    y_one_hot = torch.zeros(data.size()[0], data.size()[-1], data.size()[1], bins).scatter_(3, y_tensor.long(), 1)

    ret_indexes = temp.transpose(2,1)
    ret_histogram = y_one_hot.sum(2).transpose(2,1)

    return ret_histogram, ret_indexes


# Histogram2d

In [9]:
def np_solution2d(x, nbins):
    histograms2d = np.zeros((batch_size, nbins, feature_size))
    for i in range(batch_size):
        for j in range(feature_size):
            histograms2d[i,:,j] = np.histogram(x[i,:,j], bins=nbins, range=(0,1))[0]
    return histograms2d

def my_histogram2d(x, nbins):
    histograms2d = torch.zeros((batch_size, nbins, feature_size))
    histograms2d_idx = torch.zeros(x.shape)
    for i in range(batch_size):
        for j in range(feature_size):
            histograms2d[i,:,j],histograms2d_idx[i,:,j] = histogram(x[i,:,j],bins=nbins)
            
    return histograms2d, histograms2d_idx
def my_histogram2d_vec(x, nbins):
    histogram2d, histogram2d_indexes = histogram2d_vectorize(x, nbins)
    return histogram2d, histogram2d_indexes

np_hist2d = np_solution2d(test_data, nbins)
my_hist2d, my_hist2d_idx = my_histogram2d(torch.from_numpy(test_data), nbins)
my_hist2d_vec, my_hist2d_vec_idx = my_histogram2d_vec(torch.from_numpy(test_data), nbins)

assert(np.all(np_hist2d==my_hist2d))
assert(np.all(np_hist2d==my_hist2d_vec))
assert(np.all(my_hist2d_idx.float()==my_hist2d_vec_idx.float()))


# Testing Vectorizing Backward Function

In [10]:
# shows pytorch can boardcast
bs, bn = 5, 6
idx = torch.zeros(bs,bn).cuda()+0.1
w = torch.ones(bn).cuda()
hist = torch.zeros(bs,bn).cuda()+2
# print(I*y/z)
# values = grad_output.data*weight/histograms
v = idx*w/hist
for i in range(bs):
    for j in range(bn):
        assert(v[i,j]==(idx[i,j]*w[j]/hist[i,j]))

# for i in range(input.size()[0]): #batch_size
#     grad_input[i] = torch.index_select(values[i], 0, histograms_indexes[i].long().view(-1)).view(grad_input[i].size())
#     for a in range(input.size()[1]): #num_of_points
#         for b in range(input.size()[2]):
#             bin_number = int(histograms_indexes[i,a,b])

#             grad_input3[i,a,b] = grad_output.data[i,bin_number]*weight[bin_number]/histograms[i,bin_number] #TODO: divide by number of element in the bin
#             print(values[i,bin_number]==grad_input3[i,a,b])
        

# indices: same size as output; the corresponding index value in x 
# x : the values to be assign
# index_select: given x (value to be drawn from), dim, vector of indexes
#               return a vector of values( x[indices[i] for i in indexes])

# 1d case
bs, seq, fs, bn = 3, 5, 4, 4
indices = torch.LongTensor(bs, seq, fs).random_(0,bn)
indices_orig = indices.clone()
# np equivalent
i3 = torch.from_numpy(np.repeat(np.arange(bs),seq*fs)*bn).view(indices.shape)
# torch version
i2 = torch.arange(0, bs*bn, step=bn).unsqueeze(-1).expand(-1,seq*fs)

# print(i3)#.repeat(seq*fs)*bn
i2 = i2.view(indices.shape).long()

indices = indices + i2
grad_output = torch.randn(bs,bn)*10
grad_output = grad_output.int()

ans = torch.index_select(grad_output.view(-1), 0, indices.view(-1)).view(bs,seq,fs)

for i in range(bs):
    for x in range(fs):
        for y in range(seq):
            bin_number = indices_orig[i,y,x]

            assert(ans[i,y,x] == grad_output[i,bin_number])
            

# 2d case
# batch x feature x seq
bs, seq, fs, bn = 3,5,3,4
indices = torch.LongTensor(bs, seq, fs).random_(0,bn)
indices_orig = indices.clone()

indices = indices.transpose(2,1)
# np version
i2 = torch.from_numpy(np.repeat(np.arange(bs*fs),seq)*bn).view(indices.shape)
# torch version
i2 = torch.arange(0, bs*fs*bn, step=bn).unsqueeze(-1).expand(-1,seq)
i2 = i2.view(indices.shape).long()
# batch x feature x bins
grad_output = torch.randn(bs,bn,fs).transpose(2,1)*10
grad_output = grad_output.int()
weight = (torch.randn(bn,fs)*10).int()
indices = indices + i2

ans = torch.index_select(grad_output.view(-1), 0, indices.view(-1)).view(indices.shape)

grad_output = grad_output.transpose(2,1)
ans = ans.transpose(2,1)


for i in range(bs):
    for x in range(fs):
        for y in range(seq):
            bin_number = indices_orig[i,y,x]
            assert(ans[i,y,x]==grad_output[i,bin_number,x])
            

# psudo code for grad output
# for i in range(batch_size):
#     for x in range(feature_size):
#         for y in range(seq):
#             bin_number = hist_index[i,y,x]
#             grad_input[i,y,x] = grad_output[i,bin_number,x]*weight[bin_number,x]/histogram2d[i,bin_number,x] 




# Histogram 1d Pytorch Function and Module

In [27]:
class HistogramFunction(torch.autograd.Function):
    """
    assume input is already normalized from 0 to 1
    assume input size is batch_size, num_points, feature_size
    
    """
    
    """
    PARAM: input, weight, bins
    
    input: batch_size, num_points, feature_size
    weight:  (TBD) size of input OR size of bins
    bins: 10 by defaulted
    """
    @staticmethod
    def forward(ctx, input, weight, bins=10):
        batch_size = input.size()[0]
        histograms = torch.zeros(batch_size, bins).cuda()
        histograms_indexes = torch.zeros(input.size()).cuda()
        
        histograms, histograms_indexes = histogram_vectorize(input, bins=bins)
        histograms = histograms.cuda()
        histograms_indexes = histograms_indexes.cuda()
        
        histograms = histograms* weight
        
        #non-vec version
#         for i in range(batch_size):
#             # assert sum is the same as the count
#             histograms[i,:], histograms_values[i,:] = histogram(input[i], bins=bins)
#             histograms[i,:] = histograms[i,:] * weight
        
        ctx.save_for_backward(input, weight)
        ctx.histograms_indexes = histograms_indexes
        ctx.histograms = histograms
        return histograms
    
    @staticmethod
    def backward(ctx, grad_output):
        input, weight= ctx.saved_tensors
        histograms_indexes = ctx.histograms_indexes
        histograms = ctx.histograms
        grad_input = torch.zeros(input.size()).cuda()


        """
        the gradient at batch item i, point a, element b:
          grad_output[i, bin_number]*weight[bin_number]/(histogram[i,bin_number])
        """
#         values = grad_output.data*weight/histograms

        
        for i in range(input.size()[0]): #batch_size
#             grad_input[i] = torch.index_select(values[i], 0, histograms_indexes[i].long().view(-1)).view(grad_input[i].size())
            for a in range(input.size()[1]): #num_of_points
                for b in range(input.size()[2]):
                    bin_number = int(histograms_indexes[i,a,b])
                    grad_input[i,a,b] = grad_output.data[i,bin_number]*weight[bin_number]/histograms[i,bin_number]
#                     if (values[i,bin_number] - grad_output.data[i,bin_number]*weight[bin_number]/histograms[i,bin_number])>1e-5:
#                         print("HI")
#             assert(np.all(grad_input[i]-grad_input3[i]<=1e-4))
        
    
        # vectorize version:: ######################
        # values is the grad_output
        # histograms_values are the indices
#         bs, seq, fs, bn = input.size()[0], input.size()[1], input.size()[2], weight.size()[0]
#         # np equivalent
# #         idx_cat = torch.from_numpy(np.repeat(np.arange(bs),seq*fs)*bn).view(indices.shape)
#         # torch version
#         idx_cat = torch.arange(0, bs*bn, step=bn).cuda().unsqueeze(-1).expand(-1,seq*fs)

#         idx_cat = idx_cat.view(histograms_indexes.shape).int()

#         histograms_indexes = histograms_indexes + idx_cat

#         grad_input = torch.index_select(values.view(-1), 0, histograms_indexes.long().view(-1)).view(bs,seq,fs)
        ####################
        
        grad_weight = (grad_output.data*histograms).sum(0)/bs
        
#         print(grad_input)
        
        return Variable(grad_input), Variable(grad_weight), None
        

class Histogram(nn.Module):
    def __init__(self, bins=10):
        super(Histogram, self).__init__()
        self.bins = bins

        # nn.Parameter is a special kind of Variable, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        self.weights = nn.Parameter(torch.Tensor(bins).cuda())

        # Not a very smart way to initialize weights
        self.weights.data.uniform_(0, 1)
        

    def forward(self, input):
        
        return HistogramFunction.apply(input, self.weights, self.bins)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'bins={}'.format(self.bins)
        

# 1d Gradcheck

In [28]:
input = Variable(rangeNormalize(torch.randn(2,3,2).cuda().float()),requires_grad=True)

bins = 4
weights = Variable(torch.randn(bins).cuda().float(),requires_grad=True)
weights.data.fill_(1.)
res = gradcheck(HistogramFunction.apply, (input, weights, bins), eps=1e-6, atol=1e-4)
print(res)


(0 ,.,.) = 
  0  0
  0  0
  0  1

(1 ,.,.) = 
  0  0
  0  0
  0  0
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0  0
  0  0
  0  1

(1 ,.,.) = 
  0  0
  0  0
  0  0
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0  0
  0  0
  0  0

(1 ,.,.) = 
  0  0
  0  0
  0  0
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0  0
  0  0
  0  0

(1 ,.,.) = 
  0  0
  0  0
  0  0
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0.2500  0.2500
  0.2500  0.0000
  0.2500  0.0000

(1 ,.,.) = 
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0.2500  0.2500
  0.2500  0.0000
  0.2500  0.0000

(1 ,.,.) = 
  0.0000  0.0000
  0.0000  0.0000
  0.0000  0.0000
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0  0
  0  1
  0  0

(1 ,.,.) = 
  0  0
  0  0
  0  0
[torch.cuda.FloatTensor of size 2x3x2 (GPU 0)]


(0 ,.,.) = 
  0  0
  0  1
  0  0

(1 ,.,.) = 
  0  0
  0  0
  0  0


RuntimeError: for output no. 1,
 numerical:(
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0
[torch.FloatTensor of size 12x8]
, 
 0.9835  0.0000  0.0000  0.0000  0.9835  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000  2.9802  0.0000  0.0000
 0.0000  0.0000  3.9339  0.0000  0.0000  0.0000  0.9835  0.0000
 0.0000  0.0000  0.0000  0.9835  0.0000  0.0000  0.0000  0.9835
[torch.FloatTensor of size 4x8]
)
analytical:(
    0     0     0     0     0     0     

# Histogram 2d Pytorch Function and Module

In [16]:
class Histogram2dFunction(torch.autograd.Function):
    """
    assume input is already normalized from 0 to 1
    assume input size is batch_size, num_points, feature_size
    
    """
    
    """
    PARAM: input, weight, bins
    
    input: batch_size, num_points, feature_size
    weight:  (TBD) size of input OR size of bins
    bins: 10 by defaulted
    """
    @staticmethod
    def forward(ctx, input, weight, bins=10):
#         batch_size = input.size()[0]
#         feature_size = input.size()[-1]
#         histograms = torch.zeros(batch_size, feature_size, bins).cuda()
#         histograms_indexes = torch.zeros(input.size()).cuda()
        
        histograms, histograms_indexes = histogram2d_vectorize(input, bins=bins)
        
        histograms = histograms.cuda().double()
        histograms_values = histograms_indexes.cuda().double()
        # hist size: batch, bins, feature; weight size: bins, feature
        histograms = histograms*weight
        
        ctx.save_for_backward(input, weight)
        ctx.histograms_indexes = histograms_indexes
        ctx.histograms = histograms
        return histograms
    
    @staticmethod
    def backward(ctx, grad_output):
        input, weight= ctx.saved_tensors
        histograms_indexes = ctx.histograms_indexes
        histograms = ctx.histograms

        grad_input = torch.zeros(input.size()).cuda()

        """
        the gradient at batch item i, point a, element b:
          grad_output[i, bin_number]*weight[bin_number]/(LEN(histogram[i,bin_number]))
        """
        values = grad_output.data*weight/histograms
#         for i in range(input.size()[0]): #batch_size
#             grad_input[i] = torch.index_select(values[i], 0, histograms_values[i].long().view(-1)).view(grad_input[i].size())

#             for a in range(input.size()[1]): #num_of_points
#                 for b in range(input.size()[2]):
#                     bin_number = int(histograms_values[i,a,b])
                    
#                     grad_input[i,a,b] = grad_output.data[i,bin_number]*weight[bin_number]/histograms[i,bin_number] #TODO: divide by number of element in the bin
        # values = grad_output
        # histogram_indexes = indices
        bs, seq, fs, bn = input.size()[0], input.size()[1], input.size()[2], weight.size()[0]
        
        histograms_indexes = histograms_indexes.transpose(2,1)
        # np version
#         i2 = torch.from_numpy(np.repeat(np.arange(bs*fs),seq)*bn).view(indices.shape)
        # torch version
        idx_cat = torch.arange(0, bs*fs*bn, step=bn).cuda().unsqueeze(-1).expand(-1,seq)
        idx_cat = idx_cat.view(histograms_indexes.shape).int()
        # batch x feature x bins
        values = values.transpose(2,1)
        histograms_indexes = histograms_indexes + idx_cat

        grad_input = torch.index_select(values.contiguous().view(-1), 0, histograms_indexes.long().contiguous().view(-1)).view(histograms_indexes.shape)
        
        grad_input = grad_input.transpose(2,1)
        
        grad_weight = (grad_output.data*histograms).sum(0)/bs
        
        
        return Variable(grad_input), Variable(grad_weight), None
        

class Histogram2d(nn.Module):
    def __init__(self, feature_size, bins=10):
        super(Histogram2d, self).__init__()
        self.bins = bins
        self.feature_size = feature_size

        # nn.Parameter is a special kind of Variable, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        self.weights = nn.Parameter(torch.Tensor(bins, feature_size).cuda())

        # Not a very smart way to initialize weights
        self.weights.data.uniform_(0, 1)
        

    def forward(self, input):
        
        return Histogram2dFunction.apply(input, self.weights, self.bins)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'bins={}'.format(self.bins)
        

# 2d Gradcheck

In [18]:
input = Variable(rangeNormalize(torch.randn(3,10,5).cuda().double()),requires_grad=True)

bins = 3
weights = nn.Parameter(torch.randn(bins, 5).cuda().double())
weights.data.fill_(1.)
res = gradcheck(Histogram2dFunction.apply, (input, weights, bins), eps=1e-6, atol=1e-4)
print(res)

RuntimeError: for output no. 0,
 numerical:(
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.FloatTensor of size 150x45]
, 

Columns 0 to 12 
    3     0     0     0     0     0     0     0     0     0     0     0     0
    0     1     0     0     0     0     0     0     0     0     0     0     0
    0     0     3     0     0     0     0     0     0     0     0     0     0
    0     0     0     4     0     0     0     0     0     0     0     0     0
    0     0     0     0     1     0     0     0     0     0     0     0     0
    0     0     0     0     0     6     0     0     0     0     0     0     0
    0     0     0     0     0     0     7     0     0     0     0     0     0
    0     0     0     0     0     0     0     7     0     0

# Sample Networks

In [ ]:
class myNet(nn.Module):
    def __init__(self, in_features, H, bins=10):
        super(myNet, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 3, 1)
        self.bn1 = nn.BatchNorm1d(3)
        self.bins = bins
        self.his = Histogram(bins=bins)
        self.fc1 = nn.Linear(bins, 2)
    
    def forward(self, x):
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = x.transpose(2,1)
        x = rangeNormalize(x)
        x = self.his(x)
        x = self.fc1(x)
        return x
    
class myNet2d(nn.Module):
    def __init__(self, in_features, H, bins=10):
        super(myNet2d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 5, 1)
        self.bn1 = nn.BatchNorm1d(5)
        self.bins = bins
        self.his = Histogram2d(5, bins=bins)
        self.fc1 = nn.Linear(bins*5, 2)
        
    
    def forward(self, x):
        x = x.transpose(2,1)
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = x.transpose(2,1)
        x = rangeNormalize(x)
        x = self.his(x)
#         print(x.size())
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        
        
        return x

# Running network with dummy data

In [ ]:

t = time.time()

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, num_points, label_size, input_dim, num_bins= 32, 3, 2, 3, 3

# Create random Tensors to hold input and outputs, and wrap them in Variables.
x = Variable(torch.randn(N, num_points, input_dim).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, label_size).type(dtype).cuda(), requires_grad=False)

# Create random Tensors for weights, and wrap them in Variables.
# w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
# w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

model = myNet2d(num_points, input_dim, bins=num_bins).cuda()
optimizer = optim.SGD(model.parameters(), lr = 1e-6, momentum=0.8)


for t in range(500):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    
    optimizer.zero_grad()
    # Forward pass: compute predicted y using operations on Variables; we compute
    # ReLU using our custom autograd operation.
    
    y_pred = model(x.cuda())

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data[0])
    if(loss.data[0]==float('inf')):
        break

    # Use autograd to compute the backward pass.
    loss.backward()
    optimizer.step()
    

print(time.time() - t)
